# Notes : étapes d'un projet de ML


|N°|Step|Stage|
|--------|-----------------------|------------|
|1. |Frame the problem and look at the big picture.||
|2. |Get the data.||
|3. |Explore the data to gain insights.||
|4. |Prepare the data to better expose the underlying data patterns to Machine Learning algorithms.||
|5. |Explore many different models and shortlist the best ones.||
|6. |Fine-tune your models and combine them into a great solution.||
|7. |Present your solution.||
|8. |Launch, monitor, and maintain your system.||

## 1. Frame the problem and look at the big picture

#### - déjà donné dans nos exemples  

#### - définir la variable target (cas général)

## 2. Get the data

#### - définir la target (valeurs réelles)

#### - dataset déjà donné dans nos exemples

>`pd.read_XXX()`  
>`sklearn.datasets.load_XXX`  

#### - générer des données

> `np.linspace()`  
> `np.logspace()`

#### - clean surtout en enlevant les NA ou en les recodant (cf partie d'après)

>`pd.dropna()`  
>`pd.fillna()`  
>`pd.replace()`  
>`pd.rename()`  
>`pd.drop()`  
>`sklearn.impute.SimpleImputer`  

#### - case_when en pandas/numpy :  

<font size=2>
<code>
<pre>
conditions=[df['colonne']=='string1',   
            df['colonne']=='string2',  
            df['colonne']=='string3'  
            ]  
choices=[1, 2, 3]  
df_temp['nouvellecolonne']=np.select(conditions, choices, np.nan).astype(float)  
</pre></code></font>  

#### - Supprimer selon un type de donnée  

> `df.select_dtypes(include='number')`


## 3. Explore the data to gain insights

#### - regarder la nature des features, la présence de NA

> `df.describe()`  
> `df.dtypes`  
> `df.info()`  
> Fonction pour afficher les attributs des variables + la proportion de NA : 
<font size=2 color="grey">
<code>
<pre> 
def table_variable_attributes(df : pd.DataFrame) -> pd.DataFrame:  
    """Création table avec attributs des variables et % de valeurs manquantes"""  
    df_temp = (pd.DataFrame(  
        {  
            'variable' : df.isna().describe(include="all").columns,  
            'count' : df.isna().describe(include="all").loc['count'].astype(float),  
            'unique' : df.isna().describe(include="all").loc['unique'].astype(float),  
            'top' : df.isna().describe(include="all").loc['top'].astype(str),  
            'freq_top' : df.isna().describe(include="all").loc['freq'].astype(float)  
            # "unique" dit s'il y a des na ou pas dans la colonne (1=pas de NA, 2 = présence de Na)  
            # "count" nous donne le nb de lignes total  
            # "freq" nous donne le nb de lignes du "top"  
        }  
    )  
    .assign(percent_top=lambda df : df['freq_top']/df['count'])  
    .reset_index()  
    .assign(percent_na = lambda df : [df.loc[:,'percent_top'][row] if df['top'][row]=='True' else 1-df['percent_top'][row] for row in df.index] )  
    .drop(columns=['percent_top', 'index', 'unique', 'top', 'freq_top'])  
    .assign(percent_na=lambda df : round(df['percent_na']*100, 0),  
            count=lambda df : df['count'].astype(int))  
    .assign(type_variable = df.dtypes.astype(str).tolist())  
    )  
    return df_temp  
    </pre> </code> </font>

#### - histogrammes, boxplot, pairplots

> `seaborn.histplot()`  
> `seaborn.boxplot()`  
> `seaborn.pairplot()`  
> `seaborn.lineplot()`  
> Fonction pour afficher des countplot pour chaque variable :  
<code> <font size=2>
f, axes = plt.subplots(4,3,figsize=(17,13), sharex=False)  
for i, feature in enumerate(df_cat_var_list):  
    sns.countplot(data=df, x=feature, ax=axes[i%4, i//4])  
    axes[i%4, i//4].tick_params(axis='x', labelrotation = 45)  
      
plt.tight_layout()  
plt.show()  
</font>
</code>

#### - caractéristiques des variables/attributs :  

|Characteristic|Comments|
|--------------|----------------------------------------|
|Name||
|Type| categorical, int/float, bounded/unbounded, text, structured, etc.|
|% of missing values||
|Noisiness and type of noise| stochastic, outliers, rounding errors, etc. |
|Usefulness for the task||
|Type of distribution| Gaussian, uniform, logarithmic, etc.|

#### - séparer la target des autres  

#### - Identifier des transformations possibles (log, ln, exp, sqrt, etc)


## 4. Prepare the data to better expose the underlying data patterns to ML

#### - Recoder certaines variables catégorielles à la main, ou en créant des nouvelles variables booléennes

> `sklearn.preprocessing.OrdinalEncoder`   
> `sklearn.preprocessing.OneHotEncoder`  
> `pd.get_dummies()`  
> enlever les individus appartenant à une seule catégorie d'une variable et des mettre dans 'Other'  

<code> <font size=2>
def modification_categories_occurences_uniques(df : pd.DataFrame, nom_colonne:str) -> pd.DataFrame:  
    """modification d'un df avec ancien nom de catégorie et nouveau nom pour une colonne donnée"""  
    replacing_df = (pd.DataFrame(  
        df  
        .groupby(nom_colonne)  
        .count().loc[:, 'lat'])  
        .rename(columns={'lat':'count'})  
        .reset_index(names='category_old')  
        .assign(category_new = lambda df : ['Other' if df['count'][row]<=5 else df['category_old'][row] for row in df.index] )  
        .drop(columns='count')  
    )  
    df_temp = (df  
               .replace(to_replace=list(replacing_df.category_old), value=list(replacing_df.category_new))  
                )  
    return df_temp  
</font> </code>

#### - Séparer un sous-ensemble de train et un autre de test /!\ vérifier pour les variables catégorielles si les datasets sont balanced (surtout pour y mais aussi d'autres features qui peuvent induire un biais comme le sexe)

> `sklearn.model_selection.train_test_split()` *(stratify)*  
> 

#### - Normaliser les données (standardisation, mix-max, etc) /!\ on fit_transform les données de train et on transforme juste les données de test

> `sklearn.preprocessing.StandardScaler()`  

#### - Regarder les corrélations entre variables  

> `df.corr()`

#### - Sélectionner les features les plus importants 

>

#### - Exporter un dataset clean intermédiaire 

> `df.to_XXX()`

## 5. Explore many different models and shortlist the best ones

#### - choisir un modèle : classification ou régression?  

<img src='https://cdn.ttgtmedia.com/rms/onlineImages/enterpriseai-machine_learning_models_cheat_sheet-f.png'/>

**Régression**  
>`sklearn.linear_model.LinearRegression`  
>`sklearn.tree.DecisionTreeRegressor`  
>`sklearn.ensemble.RandomForestRegressor`  
>`sklearn.svm.LinearSVC`  

**Classification**  
> `sklearn.neighbors.KNeighborsClassifier()`
> `sklearn.svm.SVC()`  
> `sklearn.linear_model.SGDClassifier()`
> `sklearn.tree.DecisionTreeClassifier()`
> `sklearn.ensemble.RandomForestClassifier()`

#### - définir un score pour évaluer la qualité du modèle (MAE, MSE, RMSE pour régression, F1, accuracy, ROC, pour classification)

>`sklearn.metrics.XXX`  

#### - évaluer un modèle par cross-validation (cv=5 c'est bien)

> `sklearn.model_selection.cross_validate()`  
> `sklearn.model_selection.cross_val_score()`  
> `sklearn.model_selection.cross_val_predict()`


#### - comparer différents modèles  

#### - faire un pipeline  

> `sklearn.pipeline.Pipeline`  
> `sklearn.compose.ColumnTransformer`  

## 6. Fine-tune your models and combine them into a great solution

#### - connaître les hyperparamètres des modèles  

*Ressources*   
https://stackabuse.com/understanding-svm-hyperparameters/


#### - courbes d'apprentissage (learning curve)

>`sklearn.model_selection.learning_curve()`  
>`sklearn.model_selection.LearningCurveDisplay()`  

#### - courbes de validation (validation curve)

> `sklearn.model_selection.validation_curve`  
> `sklearn.model_selection.ValidationCurveDisplay`  


#### - grid search  

> `sklearn.model_selection.GridSearchCV`


## 7. Present your solution

- voir dataviz, site https://www.data-to-viz.com/  
- voir graphviz pour afficher/exporter arbre de décision  

>`sklearn.tree.export_graphviz()`  
>`graphviz.Source()`  
>`graph.render()`  
>`matplotlib.image.imread()`  
>`matplotlib.image.imshow()`  
>`sklearn.tree.plot_tree`  

## 8. Launch, monitor and maintain your system